# Collaborative Planning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, LinearConstraint

In [ ]:
# Flightroom dimensions (meters)
L = 16  # -8 to 8
W = 6   # -3 to 3


N = 4  # Number of robots
n = 2  # State dimension


bot_positions = np.zeros((N, n))
start_positions = np.array([[-6.0, 2.0],
                            [-5.0, 1.0],
                            [-5.0, -1.0],
                            [-6.0, -2.0]])
goal_positions = np.array([[6.0, 2.0],
                           [5.0, 1.0],
                           [5.0, -1.0],
                           [6.0, -2.0]])

plt.scatter(x=start_positions[:,0], y=start_positions[:,1])
plt.scatter(x=goal_positions[:,0], y=goal_positions[:,1], c='green', marker='X')
plt.axis('equal')
plt.show()

## Formation cost
* Greedy - each robot tries to maximize its own geometry, individual cost for each agent
    * GDOP per robot
* Colab - each robot tries to maximize geometry of team, overall cost for whole team

In [ ]:
def formation_cost(bot_positions):
    """Per-robot formation cost
    """
    N = len(bot_positions)
    costs = []

    for i, bot in enumerate(bot_positions):
        LOS_vecs = np.delete(bot_positions, i, axis=0) - bot
        LOS_vecs = LOS_vecs / np.linalg.norm(LOS_vecs, axis=1)[:,None]
        A = np.hstack((LOS_vecs, np.ones((N-1, 1))))
        Q = np.linalg.inv(A.T @ A)
        GDOP = np.sqrt(np.trace(Q))
        costs.append(GDOP)

    return costs

In [ ]:
def goal_cost(bot_positions, goal_positions):
    """Per-robot goal cost
    """
    dists = np.linalg.norm(bot_positions - goal_positions, axis=1)
    return dists

In [ ]:
def total_cost(dx, x, goal, lmbda):
    """Total cost objective for planning
    """
    dx = dx.reshape(x.shape)
    goal_costval = np.sum(goal_cost(x + dx, goal))
    formation_costval = np.sum(formation_cost(x + dx))
    # print(f"goal: {goal_costval}, formation: {formation_costval}")
    return goal_costval + lmbda * formation_costval

In [ ]:
def colab_planner(curr, goal, lmbda=1.0):
    """Centralized collaborative planner

    Takes in current robot positions and goal positions, and outputs 
    waypoints for next iteration

    Each waypoint has a max distance from previous position

    Goal cost + Formation cost
    
    Goal cost: distance to goal
    Formation cost: condition number of geometry matrix

    Lambda (tradeoff term): weight goal cost higher closer to goal

    """    
    N, n = curr.shape
    D_MAX = 0.2

    cons = LinearConstraint(np.eye(N * n), -D_MAX, D_MAX)
    res = minimize(total_cost, np.zeros(N * n), args=(curr, goal, lmbda), constraints=cons)
    dx = res.x

    return dx.reshape(N, n)

In [ ]:
def lmbda_map(x):
    return 1 - 1 / (x + 1) 

In [ ]:
lmbda_map(10)

In [ ]:
x = start_positions.copy()
x_hist = []
max_iters = 100

for i in range(max_iters):
    dists = np.linalg.norm(x - goal_positions, axis=1)
    if np.all(dists < 0.5):
        break
    lmbda = 0.01 * lmbda_map(0.1 * np.sum(dists))
    print(np.sum(dists), lmbda)

    dx = colab_planner(x, goal_positions, lmbda=1.0)
    x += dx
    x_hist.append(x.copy())

x_hist = np.stack(x_hist)

In [ ]:
for i in range(N):
    plt.scatter(x=x_hist[:,i,0], y=x_hist[:,i,1])
plt.axis('equal')
plt.show()